In [1]:
from bs4 import BeautifulSoup

import urllib.request

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import re
game_weeks = pd.read_csv("~\\Data.csv")
game_weeks.head()

,Year,Week,URL,Unnamed: 3
0,2019,1,https://www.espn.com/nfl/story/_/id/27155205/w...,NaN
1,2019,2,https://www.espn.com/nfl/story/_/id/27593208/w...,NaN
2,2019,3,https://www.espn.com/nfl/story/_/id/27646341/w...,NaN
3,2019,4,https://www.espn.com/nfl/story/_/id/27691599/w...,NaN
4,2019,5,https://www.espn.com/nfl/story/_/id/27752831/w...,NaN


In [3]:
print(len(game_weeks))

38


In [4]:
all_writeups = []
all_predicted_score = []
all_high_or_low = []

In [5]:
for j in range(len(game_weeks)):
    #print(game_weeks['URL'][i])
    response = urllib.request.urlopen(game_weeks['URL'][j])
    html = response.read()
    soup = BeautifulSoup(html,"html5lib")

    write_ups = soup.find_all('p')

    team_write_ups = []
    for tag in write_ups:
        write_up = tag.text.strip()
        check_if_in = ["for: ", "d prediction: ", "know: ", "stake: ", "fantasy: ", "nugget: "]

        if any(x in write_up for x in check_if_in):
            team_write_ups.append(write_up)
    
    predicted_scores = soup.find_all('strong')
    week_predicted_scores = []
    for tag in predicted_scores:
        prediction = tag.text.strip()
        if "pick:" in prediction:
          week_predicted_scores.append(prediction)
        
    week_writeups_dataframe = []
    combine_writeups_from_same_game = ""
    count = 1
    match = 0
    for i in range(len(team_write_ups)):
        #print(team_write_ups[i])
        combine_writeups_from_same_game = combine_writeups_from_same_game + " " + team_write_ups[i]
        if ("nugget: " in team_write_ups[i]):
            #print("\n")
            #print("NEW MATCH WRITEUP")
            #print("-----------------")
            week_writeups_dataframe.append(combine_writeups_from_same_game)
            combine_writeups_from_same_game = ""
            match += 1
        count += 1
        
    predictions = soup.find_all('p')
    week_predictions = []
    for tag in predictions:
        prediction = tag.text.strip()
        if "pick:" in prediction:
          week_predictions.append(prediction)
        
    sliced_week_predictions = []
    for prediction in week_predictions:
        count = 0
        i = 0
        for letter in prediction:
            if letter == "\n":
                count += 1
            if count == 2:
                sliced = prediction[0:i]
                sliced_week_predictions.append(sliced)
                #print(i, " ", sliced)
                break
            i += 1
    
    weekly_scores = []
    for prediction in sliced_week_predictions:
        weekly_scores.append(re.findall(r'\b\d+\b', prediction))
    int_weekly_scores = []
    for score in weekly_scores:
        int_weekly_scores.append([int(s) for s in score])
        
    average_predicted_score_game = []
    for score in int_weekly_scores:
        predictOne = (score[0] + score[1]) / 2
        predictTwo = (score[2] + score[3]) / 2
        averagePredict = (predictOne + predictTwo) / 2
        #print(averagePredict)
        average_predicted_score_game.append(averagePredict)
        
    high_or_low = []
    for score in average_predicted_score_game:
        if score < 23.6:
            high_or_low.append("LOW")
        else:
            high_or_low.append("HIGH")
        
    #df=pd.DataFrame()
    #df.columns=['Write Up', 'Predicted Score', "High or Low"] #provides a name for the column
    #df['Write Up'] = week_writeups_dataframe
    #df['Predicted Score'] = average_predicted_score_game
    #df['High or Low'] = high_or_low
    
    all_writeups += week_writeups_dataframe
    all_predicted_score += average_predicted_score_game
    all_high_or_low += high_or_low
    
    

In [6]:
df=pd.DataFrame()
df['Write Up'] = all_writeups
df['Predicted Score'] = all_predicted_score
df['High or Low'] = all_high_or_low
print(df)

                                              Write Up  Predicted Score  \
0     What to watch for: Nobody is really sure what...            27.25   
1     What to watch for: Don't be surprised if Cam ...            25.50   
2     What to watch for: The Julio Jones-Xavier Rho...            22.25   
3     What to watch for: The unveiling of the Baker...            21.50   
4     What to watch for: The Eagles are anticipatin...            20.25   
..                                                 ...              ...   
529   What to watch for: How will the Raiders compa...            26.75   
530   What to watch for: The Bucs have survived a r...            22.00   
531   What to watch for: It's not a homecoming beca...            24.00   
532   What to watch for: Storylines abound here, wi...            26.00   
533   What to watch for: Can Seahawks quarterback G...            23.00   

    High or Low  
0          HIGH  
1          HIGH  
2           LOW  
3           LOW  
4        